In [1]:
#import dependencies
import pandas as pd
import numpy as np
import os
import gmaps
import json
import requests
from pprint import pprint
import us

#import API key
from config import gkey, weather_api_key
gmaps.configure(api_key=gkey)

#output file path for data resource files and figures
outputFilePath = os.path.join("Output-Data")

In [2]:
#mannually imported information from text file online to csv
#mannually did some cleaning with the data in excel
#we do not know web scraping yet so was unable to utilize that for infomration

In [3]:
#read the sunshine csv file into a dataframe
sunshine_df = pd.read_csv(os.path.join("..","potential_data_sets","NOAA_Sunshine_AveragePercentPossible.csv"))
sunshine_df

,CITY,STATE,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA
0,YUMA,AZ,90%,42
1,REDDING,CA,88%,10
2,LAS VEGAS,NV,85%,47
3,PHOENIX,AZ,85%,101
4,TUCSON,AZ,85%,53
...,...,...,...,...
169,POHNPEI,FM,41%,44
170,ELKINS,WV,40%,11
171,MT. WASHINGTON,NH,33%,64
172,QUILLAYUTE,WA,33%,30


In [4]:
#clean up the column names
sunshine_df = sunshine_df.rename(columns={
    "CITY":"City",
    "STATE":"State",
    "ANNUAL % AVG POSSIBLE SUNSHINE":"Annual % Average Possible Sunshine",
    "NO. OF YEARS OF DATA":"# of Data Years"
})

In [5]:
#check the datatypes
sunshine_df.dtypes

City                                  object
State                                 object
Annual % Average Possible Sunshine    object
# of Data Years                        int64
dtype: object

In [6]:
#strip the % column of the % symbol and convert to float
sunshine_df["Annual % Average Possible Sunshine"] = sunshine_df["Annual % Average Possible Sunshine"].map(lambda x: x.rstrip('%'))
sunshine_df["Annual % Average Possible Sunshine"] = sunshine_df["Annual % Average Possible Sunshine"].astype(float)
sunshine_df

,City,State,Annual % Average Possible Sunshine,# of Data Years
0,YUMA,AZ,90.0,42
1,REDDING,CA,88.0,10
2,LAS VEGAS,NV,85.0,47
3,PHOENIX,AZ,85.0,101
4,TUCSON,AZ,85.0,53
...,...,...,...,...
169,POHNPEI,FM,41.0,44
170,ELKINS,WV,40.0,11
171,MT. WASHINGTON,NH,33.0,64
172,QUILLAYUTE,WA,33.0,30


In [7]:
#credit to: https://code.activestate.com/recipes/577305-python-dictionary-of-us-states-and-territories/
#added in some territories mannually to this dictionary
#dictionary of state abbreviations:
states_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'FM': 'Federal States of Micronesia',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MH': 'Marshall Islands',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'PW': 'Palau',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [10]:
state=[]
for row in range(len(sunshine_df)):
    try:
        state.append(states_dict[sunshine_df['State'][row]])
    except:
        state.append(sunshine_df['State'][row])

cleanSunshine_df = sunshine_df.copy()
cleanSunshine_df["State"]=state
cleanSunshine_df

,City,State,Annual % Average Possible Sunshine,# of Data Years
0,YUMA,Arizona,90.0,42
1,REDDING,California,88.0,10
2,LAS VEGAS,Nevada,85.0,47
3,PHOENIX,Arizona,85.0,101
4,TUCSON,Arizona,85.0,53
...,...,...,...,...
169,POHNPEI,Federal States of Micronesia,41.0,44
170,ELKINS,West Virginia,40.0,11
171,MT. WASHINGTON,New Hampshire,33.0,64
172,QUILLAYUTE,Washington,33.0,30


In [11]:
#clean the casing of cities
cities = cleanSunshine_df['City']
cities = cities.str.title()
cleanSunshine_df['City']=cities
cleanSunshine_df

,City,State,Annual % Average Possible Sunshine,# of Data Years
0,Yuma,Arizona,90.0,42
1,Redding,California,88.0,10
2,Las Vegas,Nevada,85.0,47
3,Phoenix,Arizona,85.0,101
4,Tucson,Arizona,85.0,53
...,...,...,...,...
169,Pohnpei,Federal States of Micronesia,41.0,44
170,Elkins,West Virginia,40.0,11
171,Mt. Washington,New Hampshire,33.0,64
172,Quillayute,Washington,33.0,30


In [12]:
#google maps geocoding url
url_geocoding = "https://maps.googleapis.com/maps/api/geocode/json?"

#find the latitude and longitude of the cities
lat = []
lng = []

#define countrycode
currentCountry = "US"

#loop through the cities_df to retrieve the weather information
for index, row in cleanSunshine_df.iterrows():
    currentCity = row["City"]
    currentState = row["State"]
    target_url = f"{url_geocoding}address={currentCity},{currentState}&region={currentCountry}&key={gkey}"
    response = requests.get(target_url).json()
    
    try:
        lat.append(response['results'][0]['geometry']['location']['lat'])
        lng.append(response['results'][0]['geometry']['location']['lng'])
        print(f"Processeing Record | {currentCity}")
    except:
        lat.append(np.nan)
        lng.append(np.nan)
        print(f"City not found. Skipping {currentCity}...")

#update latitude and longitude to actually match the city
cleanSunshine_df["Latitude"] = lat
cleanSunshine_df["Longitude"] = lng

Processeing Record | Yuma
Processeing Record | Redding
Processeing Record | Las Vegas
Processeing Record | Phoenix
Processeing Record | Tucson
Processeing Record | El Paso
Processeing Record | Fresno
Processeing Record | Reno
Processeing Record | Flagstaff
Processeing Record | Sacramento
Processeing Record | Albuquerque
Processeing Record | Key West
Processeing Record | Pueblo
Processeing Record | Midland-Odessa
Processeing Record | Roswell
Processeing Record | Amarillo
Processeing Record | Ely
Processeing Record | Los Angeles
Processeing Record | Lake Charles
Processeing Record | Lubbock
Processeing Record | North Little Rock
Processeing Record | Grand Junction
Processeing Record | Honolulu
Processeing Record | Johnston Island
Processeing Record | Abilene
Processeing Record | Dodge City
Processeing Record | Miami
Processeing Record | Milford
Processeing Record | Wake Island
Processeing Record | Denver
Processeing Record | Lander
Processeing Record | Oklahoma City
Processeing Record | 

In [13]:
#export results to csv file
cleanSunshine_df.to_csv(os.path.join(outputFilePath,"cleanSunshine.csv"),index=False)

cleanSunshine_df

,City,State,Annual % Average Possible Sunshine,# of Data Years,Latitude,Longitude
0,Yuma,Arizona,90.0,42,32.692651,-114.627692
1,Redding,California,88.0,10,40.586540,-122.391675
2,Las Vegas,Nevada,85.0,47,36.169941,-115.139830
3,Phoenix,Arizona,85.0,101,33.448377,-112.074037
4,Tucson,Arizona,85.0,53,32.222607,-110.974711
...,...,...,...,...,...,...
169,Pohnpei,Federal States of Micronesia,41.0,44,6.854125,158.262382
170,Elkins,West Virginia,40.0,11,38.925940,-79.846735
171,Mt. Washington,New Hampshire,33.0,64,44.270585,-71.303272
172,Quillayute,Washington,33.0,30,47.943130,-124.542435


In [14]:
#clean the data set of null values
sunshineLocations_df = cleanSunshine_df.dropna().reset_index(drop=True)
sunshineLocations_df

,City,State,Annual % Average Possible Sunshine,# of Data Years,Latitude,Longitude
0,Yuma,Arizona,90.0,42,32.692651,-114.627692
1,Redding,California,88.0,10,40.586540,-122.391675
2,Las Vegas,Nevada,85.0,47,36.169941,-115.139830
3,Phoenix,Arizona,85.0,101,33.448377,-112.074037
4,Tucson,Arizona,85.0,53,32.222607,-110.974711
...,...,...,...,...,...,...
169,Pohnpei,Federal States of Micronesia,41.0,44,6.854125,158.262382
170,Elkins,West Virginia,40.0,11,38.925940,-79.846735
171,Mt. Washington,New Hampshire,33.0,64,44.270585,-71.303272
172,Quillayute,Washington,33.0,30,47.943130,-124.542435


In [15]:
#collect the weightsof %
sunshine = sunshineLocations_df["Annual % Average Possible Sunshine"].astype(float)

#max of % sunshine
maxSun = sunshineLocations_df["Annual % Average Possible Sunshine"].max()

#locations of sun for heat
locations = sunshineLocations_df[["Latitude","Longitude"]].astype(float)

In [20]:
#create the sunshine map
fig = gmaps.figure()

#create the heat layer
heatLayer_sunshine = gmaps.heatmap_layer(locations, weights=sunshine, max_intensity=maxSun, dissipating=True)

#add the heat layer
fig.add_layer(heatLayer_sunshine)

#display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
#find locations with sun >= 70%
topSunshine_df = sunshineLocations_df.loc[(sunshineLocations_df["Annual % Average Possible Sunshine"]>=70)]
topSunshine_df

,City,State,Annual % Average Possible Sunshine,# of Data Years,Latitude,Longitude
0,Yuma,Arizona,90.0,42,32.692651,-114.627692
1,Redding,California,88.0,10,40.586540,-122.391675
2,Las Vegas,Nevada,85.0,47,36.169941,-115.139830
3,Phoenix,Arizona,85.0,101,33.448377,-112.074037
4,Tucson,Arizona,85.0,53,32.222607,-110.974711
5,El Paso,Texas,84.0,54,31.761878,-106.485022
6,Fresno,California,79.0,46,36.737798,-119.787125
7,Reno,Nevada,79.0,45,39.529633,-119.813803
8,Flagstaff,Arizona,78.0,15,35.198284,-111.651302
9,Sacramento,California,78.0,46,38.581572,-121.494400


In [18]:
#DO NOT CHANGE CODE IN THIS CELL

#using the infobox template
info_box_template = """
<dl>
<dt>State</dt><dd>{State}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>% Sunshine</dt><dd>{Annual % Average Possible Sunshine}</dd>
</dl>
"""

#Store the dataframe row
#update with names of dataframe
sunshine_info = [info_box_template.format(**row) for index, row in topSunshine_df.iterrows()]
markerLocations = topSunshine_df[['Latitude','Longitude']]

In [19]:
#add marker layer onto heatmap
markers = gmaps.marker_layer(markerLocations,info_box_content=sunshine_info)
fig.add_layer(markers)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))